# Building RAG model chatbot using Langchain

## Importing the libraries

In [5]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

## Setting the API key

In [2]:
import os
import dotenv
dotenv.load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## Building the RAG basics

In [71]:
vectorstore = FAISS.from_texts(
    ["""
In the vibrant tapestry of imagination, there exists a character named Sasi—an individual who defies conventional norms and embodies the essence of uniqueness. Sasi, with their kaleidoscopic personality, is a fusion of creativity, intellect, and a dash of enigma.

Sasi is an avid explorer of the mind, constantly pushing the boundaries of imagination. With an insatiable curiosity, they delve into the realms of the unknown, unraveling mysteries that others might overlook. Sasi's mind is a labyrinth of ideas, a sanctuary where the extraordinary flourishes.

A polymath by nature, Sasi effortlessly navigates between the arts and sciences, seamlessly blending the analytical precision of a scientist with the boundless creativity of an artist. In the morning, Sasi may be found immersed in the intricate details of a scientific experiment, and by evening, they seamlessly transition into a world of colors, bringing canvases to life with strokes that speak volumes.

Sasi's wardrobe is an eclectic mix of styles, reflecting their ever-evolving identity. From classic elegance to avant-garde expressions, each outfit is a testament to their fluidity and adaptability. Socially magnetic, Sasi effortlessly connects with a diverse array of people, leaving an indelible impression with their charisma and genuine interest in others.

In the realm of imagination, Sasi is an icon—a symbol of boundless creativity, intellectual prowess, and the beauty that arises when one embraces the full spectrum of their identity.
     """], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

model = ChatOpenAI()

In [62]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [8]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [9]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

# Adding the chat history

In [10]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel

In [64]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in English language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [65]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [66]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [67]:
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [68]:
conversational_qa_chain.invoke(
    {
        "question": "where did harrison work?",
        "chat_history": [],
    }
)

AIMessage(content='There is no mention of a character named Harrison in the context provided.')

In [17]:
conversational_qa_chain.invoke(
    {
        "question": "What is the first letter?",
        "chat_history": [
            HumanMessage(content="Who wrote this notebook?"),
            AIMessage(content="Harrison"),
        ],
    }
)

AIMessage(content='The first letter is "H"')

## Adding Memory

In [41]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

In [42]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [72]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [57]:
inputs = {"question": "what was my third last question"}
result = final_chain.invoke(inputs)
result['answer'].content

"I'm sorry, but based on the context given, there is no information provided about Vishnu asking any questions."

In [51]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [74]:
memory.load_memory_variables({})

{'history': []}

In [24]:
inputs = {"question": "but where did he really work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Harrison really worked at Kensho.'),
 'docs': [Document(page_content='harrison worked at kensho')]}

In [73]:
memory.clear()

## Final chatbot

In [75]:
flag = True
while flag:
    ques = input("Ask a question or X to stop:")
    if ques == "X":
        break
    inputs = {"question": ques}
    result = final_chain.invoke(inputs)
    memory.save_context(inputs, {"answer": result["answer"].content})
    print(result['answer'].content)


The topic of the article is the character of Sasi and their unique and imaginative personality.
Sasi is a character who embodies uniqueness, creativity, intellect, and a blend of art and science. They are an avid explorer of the mind, a polymath, socially magnetic, and an icon in the realm of imagination.
Based on the context provided, Sasi can be considered a great person due to their unique and exceptional qualities such as creativity, intellect, curiosity, versatility, and ability to connect with others. Their ability to seamlessly navigate between different fields and their impact in the realm of imagination make them stand out as a remarkable individual.
